In [1]:
!pip install transformers[torch]
!pip install sentencepiece
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.5 MB/s eta 0:00:00


In [2]:
import random
from tabulate import tabulate
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report

# datasets
from datasets import Dataset
from datasets import DatasetDict
from datasets import load_metric
from datasets import load_dataset

# transformers
from transformers import Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import IntervalStrategy

import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from sklearn.metrics import accuracy_score, f1_score

import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from datasets.utils.logging import disable_progress_bar
from transformers import logging


disable_progress_bar()
logging.set_verbosity_error()

In [4]:
directory = "/kaggle/input/amazon-with-translated"
# directory = "/content/drive/MyDrive/MSc/NLP/nlp-project"

In [5]:
train_path = f"{directory}/train.csv"
test_path = f"{directory}/test.csv"
valid_path = f"{directory}/valid.csv"
translated_test_path = f"{directory}/amazon_translated_body_and_title_with_originals_all_stars.csv"

In [6]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
valid_df = pd.read_csv(valid_path)
translated_test_df = pd.read_csv(translated_test_path)

In [7]:
translated_test_df.rename(columns={"review_body": "review_body_original", "review_title": "review_title_original"}, inplace=True)

In [8]:
translated_test_df.rename(columns={"translated_body": "review_body", "translated_title": "review_title"}, inplace=True)

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)


cuda


In [10]:
SEED = 111

# Set the random seed for Python to SEED
random.seed(SEED)

# Set the random seed for numpy to SEED
np.random.seed(SEED)

# Set the random seed for torch to SEED
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [11]:
def verbose_print(msg, verbose=False):
    if verbose:
        print(msg)


def drop_data(df, stars, languages, columns_to_drop):
    df = df[df["stars"].isin(stars)]
    df = df[df["language"].isin(languages)]
    if len(columns_to_drop) > 0:
        df.drop(columns=columns_to_drop, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


def prepare_data(
    train, test, valid, columns_to_drop, stars=[1, 2, 3, 4], languages=["en"]
):
    train = drop_data(train, stars, languages, columns_to_drop)
    test = drop_data(test, stars, languages, columns_to_drop)
    valid = drop_data(valid, stars, languages, columns_to_drop)

    return train, test, valid


def reduce_dataset(df, stars, languages, num_of_rows_to_drop, verbose=False):
    for lang in languages:
        for star in stars:
            verbose_print(f"Language: {lang}, Stars: {star}", verbose)
            verbose_print(
                f'Number of rows before: {len(df[(df["language"] == lang) & (df["stars"] == star)])}',
                verbose,
            )
            random_indices = np.random.choice(
                df[(df["language"] == lang) & (df["stars"] == star)].index,
                num_of_rows_to_drop,
                replace=False,
            )
            df.drop(index=random_indices, inplace=True)
            verbose_print(
                f'Number of rows after: {len(df[(df["language"] == lang) & (df["stars"] == star)])}',
                verbose,
            )

    return df


def replace_mapping(df, label, mapping):
    for k, v in mapping.items():
        df[label].replace(k, v, inplace=True)
    return df


def prepare_truncation(data_df, tokenizer, m, n):
    for i, r in tqdm(data_df.iterrows(), total=len(data_df), desc="Processing reviews"):
        tokenized_row = tokenizer.tokenize(r["review_body"])
        if len(tokenized_row) > m + n:
            data_df.loc[i, "review_body"] = tokenizer.convert_tokens_to_string(
                tokenized_row[:m] + tokenized_row[-n:]
            )

    return data_df


def print_using_tabulate(data):
    table_data = []
    for key, values in data.items():
        if key != "macro avg" and key != "weighted avg":
            if isinstance(values, dict):
                row = [
                    key,
                    values["precision"],
                    values["recall"],
                    values["f1-score"],
                    values["support"],
                ]
                table_data.append(row)

    # Print the classification report using tabulate
    headers = ["Class", "Precision", "Recall", "F1-Score", "Support"]
    m_table = tabulate(table_data, headers=headers, tablefmt="psql", floatfmt=".4f")
    print(m_table)


def eval_model(trainer, test_set, target_names, label):
    predictions = trainer.predict(test_set)
    predicted_labels = predictions.predictions.argmax(axis=1)
    ground_truth_labels = test_set[label]
    classification_reports = classification_report(
        ground_truth_labels,
        predicted_labels,
        target_names=target_names,
        output_dict=True,
    )
    print_using_tabulate(classification_reports)
    print("\n\n\n")


load_accuracy = evaluate.load("accuracy")
load_f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)[
        "accuracy"
    ]
    f1 = load_f1.compute(
        predictions=predictions, references=labels, average="weighted"
    )["f1"]
    return {"accuracy": accuracy, "f1": f1}


# Preprocess function with labels
def preprocess_function(examples):
    inputs = tokenizer(examples["review_body"], truncation=True)
    inputs["labels"] = examples["stars"]
    return inputs


def train_model(
    model_name,
    num_of_labels,
    dataset,
    tokenizer,
    path_to_save,
    epochs=1,
    disable_tqdm=False,
):
    training_args = TrainingArguments(
        output_dir=path_to_save,
        warmup_steps=10000,
        optim="adamw_torch",
        num_train_epochs=epochs,
        weight_decay=1e-4,
        evaluation_strategy=IntervalStrategy.EPOCH,
        save_strategy=IntervalStrategy.EPOCH,
        metric_for_best_model="f1",
        save_total_limit=1,
        disable_tqdm=disable_tqdm,
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_of_labels
    )
    tokenized_train = dataset["train"].map(preprocess_function, batched=True)
    tokenized_validation = dataset["validation"].map(preprocess_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_validation,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer


def prepare_dataset(train_df, test_df, valid_df, translated_df):
    train_ds = Dataset.from_pandas(train_df)
    test_ds = Dataset.from_pandas(test_df)
    valid_ds = Dataset.from_pandas(valid_df)
    translated_ds = Dataset.from_pandas(translated_df)

    dataset = DatasetDict()

    dataset["train"] = train_ds
    dataset["validation"] = valid_ds
    dataset["test"] = test_ds
    dataset["translated"] = translated_ds

    return dataset


def run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    verbose=False,
    epochs=1,
    disable_tqdm=True):

    train, test, val = prepare_data(
        train_df, test_df, valid_df, columns_to_drop, stars, languages
    )
    train = reduce_dataset(train, stars, languages, rows_to_drop, verbose)
    train = replace_mapping(train, label, mapping)
    test = replace_mapping(test, label, mapping)
    val = replace_mapping(val, label, mapping)

    translated = drop_data(translated_test_df, stars, ["de", "es", "fr"], [])
    translated = replace_mapping(translated, label, mapping)

    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, num_labels=num_of_labels)
    train = prepare_truncation(train, tokenizer, M, N)
    test = prepare_truncation(test, tokenizer, M, N)
    val = prepare_truncation(val, tokenizer, M, N)
    translated = prepare_truncation(translated, tokenizer, M, N)

    dataset = prepare_dataset(train, test, val, translated)
    trainer = train_model(
        model_name,
        num_of_labels,
        dataset,
        tokenizer,
        f"./{model_name}_labels_{num_of_labels}_M_{M}_N_{N}",
        epochs=epochs,
        disable_tqdm=disable_tqdm
    )

    print(f"Results for model: {model_name}, M = {M}, N = {N}")
    print("English test set results")
    tokenized_train = dataset["test"].map(preprocess_function, batched=True)
    eval_model(trainer, tokenized_train, label_names, label)
    print()
    print("Translated to English set results")
    tokenized_train = dataset["translated"].map(preprocess_function, batched=True)
    eval_model(trainer, tokenized_train, label_names, label)

In [12]:
columns_to_drop = ['review_id', 'product_id', 'reviewer_id', 'product_category']
tokenizer = None
rows_to_drop = 30000

**BERT CASED**

**Two labels with Neutral; N=128, M=382; overall 510 tokens**

In [13]:
model_name = "bert-base-cased"
stars = [1, 2, 3, 4, 5]
languages = ["en"]
mapping = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
N = 128
M = 382
num_of_labels = 2
label = "stars"
label_names = ["Negative", "Positive"]
epochs = 1

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:03<00:00, 3843.01it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 label_names = ["Negative", "Positive"]                                                      │
│   10 epochs = 1                                                                                  │
│   11                                                                                             │
│ ❱ 12 run_and_eval(                                                                               │
│   13 │   train_df,                                                                               │
│   14 │   test_df,                                                                                │
│   15 │   valid_df,                                                                               │
│                                                                                                  │
│ in run_and_eval:219                                                                              │
│                                                                                                  │
│   216 │   translated = prepare_truncation(translated, tokenizer, M, N)                           │
│   217 │                                                                                          │
│   218 │   dataset = prepare_dataset(train, test, val, translated)                                │
│ ❱ 219 │   trainer = train_model(                                                                 │
│   220 │   │   model_name,                                                                        │
│   221 │   │   num_of_labels,                                                                     │
│   222 │   │   dataset,                                                                           │
│                                                                                                  │
│ in train_model:161                                                                               │
│                                                                                                  │
│   158 │   │   compute_metrics=compute_metrics,                                                   │
│   159 │   )                                                                                      │
│   160 │                                                                                          │
│ ❱ 161 │   trainer.train()                                                                        │
│   162 │   return trainer                                                                         │
│   163                                                                                            │
│   164                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                            

**Two labels with Neutral; N=64, M=64; overall 128 tokens**

In [ ]:
N = 64
M = 64

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**Two labels with Neutral; N=510, M=0; overall 510 tokens**

In [ ]:
N = 510
M = 0

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**Two labels with Neutral; N=0, M=510; overall 510 tokens**

In [ ]:
N = 0
M = 510

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**XLNET 2 labels with Neutrals**

In [ ]:
model_name = "xlnet-base-cased"

**Two labels with Neutral; N=128, M=382; overall 510 tokens**

In [ ]:
N = 128
M = 382

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**Two labels with Neutral; N=64, M=64; overall 128 tokens**

In [ ]:
N = 64
M = 64

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**Two labels with Neutral; N=510, M=0; overall 510 tokens**

In [ ]:
N = 510
M = 0

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

**Two labels with Neutral; N=0, M=510; overall 510 tokens**

In [ ]:
N = 0
M = 510

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)